![logo](https://lh3.googleusercontent.com/drive-viewer/AK7aPaD3ufeMCQTD1-doWtQSHK8snQjgqYdAscSL7mNuLmhVVAoDmdbuV5Z1eG_j5-vT4N64bUvfOrHjuw-3WrS532FsUSu9=s1600)

# Data Science Assessment

## Theoretical Part

### Question 1
#### You are given thousands of images like the one in the link below:
#### https://drive.google.com/file/d/1Q7ri0UcGmtsfYiAJ1hb8fI0bLdmsYMNi/view?usp=sharing
#### Describe the method you would use for getting key information from each product image.

*Your answer here*

1) Edge detection to get only the item, not the banners etc
2) OCR to get name, netweight etc
3) Object classification to get item category


### Question 2
#### One of our customers is interested in monitoring the grocery section of a direct competitor, Target, in the state of New York.
#### https://www.target.com/
#### Target is a huge website with tons of data publicly available. Suggest three valuable insights that we can be capturing by scraping the website on a daily basis.

We could get: 
1) Daily flunctuations in prices for specific core products as time series
2) Information about items on sale or with discount or out of stock (which can be used to reverse engineer their marketing strategy and logistics)
3) Details about new items on sale

## Coding Part

### Question 3
#### Here’s an example item from Target:
#### https://www.target.com/p/ocean-spray-cranberry-juice-cocktail-64-fl-oz-bottle/-/A-12935714#lnk=sametab
#### Design an SQL schema for storing item information when scraping all items from Target website.

In [5]:
# Your SQL code here
import logging

import psycopg2
from sqlalchemy import create_engine, text
from sqlalchemy.exc import IntegrityError
from sqlalchemy.engine.base import Connection
import pandas as pd

logger = logging.getLogger("database_utils")

HOST = 'localhost'
PORT = '5432'
DBNAME = 'BETTERBASKET'
USER = 'test'
PASSWORD = 'test'

def get_connection_parameters(host=None, port=None, dbname=None, user=None, password=None):
    logger.debug(f"Reading from local settings")
    if host is None:
        host = HOST
    if port is None:
        port = PORT
    if dbname is None:
        dbname = DBNAME
    if user is None:
        user = USER
    if password is None:
        password = PASSWORD


def engine_connect():
    host, port, dbname, user, password = get_connection_parameters()
    conn_string = f"postgresql://[{host}]:{port}/{dbname}?user={user}&password={password}"
    db = create_engine(conn_string)  # , pool_pre_ping=True
    conn = db.connect()
    logger.debug(f"Engine connect:Connecting to {conn} from secrets.")

    return conn


def connect_with_psycopg2(host=None, port=None, dbname=None, user=None, password=None):
    host, port, dbname, user, password = get_connection_parameters(host, port, dbname, user, password)
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=dbname,
        user=user,
        password=password)
    logger.info(f"Connecting to {conn} from {READ_DB_CREDENTIALS_FROM}.")
    return conn


def connect(host=None, port=None, dbname=None, user=None, password=None):
    try:
        conn = connect_with_psycopg2(host, port, dbname, user, password)
    except (Exception, psycopg2.DatabaseError) as error:
        logger.error(error)
        return None
        
def execute_commands(commands, target_function=None, **kwargs):
    results = None
    try:
        conn = connect()
        cur = conn.cursor()
        for command in commands:
            logger.debug(f"Executing command: {command}")
            cur.execute(command)
            if target_function is not None:
                result = target_function(cur, **kwargs)
                results.append(result)
        cur.close()
        conn.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        logger.error(error)
    finally:
        if conn is not None:
            conn.close()
        return results
        
def create_tables():
    """ create tables in the PostgreSQL database"""
    commands = (
        """
        CREATE TABLE USER_ITEMS (
            datetime TIMESTAMP WITH TIME ZONE PRIMARY KEY,
            item_id INT8,
            vendor VARCHAR(32),
            price FLOAT8,
            on_sale INT4, 
            weight FLOAT4,
            net weight FLOAT4,
            name VARCHAR(32),
            description VARCHAR(256),
            contains VARCHAR(32),
            form VARCHAR(32),
            TCIN INT8,
            UPC INT8,
        )
        """,
    )
    execute_commands(commands)

# Can also add the calorie section and the rest of the specification info 

### Question 4
#### We have scraped two online grocery storefronts, one of Whole Foods and and one of Fresh Direct, both located in 10002. You are given the lists of Wine & Beer 
#### items of each storefront in the link below.
#### https://drive.google.com/drive/folders/17W1vjy4Vi0d32ThDlJZTneH4bNAjgFxV?usp=sharing
#### Please create a Python script that reads the two lists shared, processes the data to find common items, and outputs the list of common items to an output file. 
#### The script should be executable from the terminal using the command:  python3 {script_name} {input1} {input2} {output}
#### If your script relies on external libraries or packages, include a requirements.txt file specifying the dependencies.

In [2]:
!python .\compare_lists.py .\fresh_direct.csv .\whole_foods.csv .\output.csv


DEBUG:compare_lists:List1:
              category  ...                                           item_url
0  Beer, Non-Alcoholic  ...  https://www.freshdirect.com/supergro/beer/sc/b...
1  Beer, Non-Alcoholic  ...  https://www.freshdirect.com/supergro/beer/sc/b...
2  Beer, Non-Alcoholic  ...  https://www.freshdirect.com/supergro/beer/sc/b...
3  Beer, Non-Alcoholic  ...  https://www.freshdirect.com/supergro/beer/sc/b...
4  Beer, Non-Alcoholic  ...  https://www.freshdirect.com/supergro/beer/sc/b...

[5 rows x 9 columns]
DEBUG:compare_lists:List2:
                                           name  ...  serving_size_uom
0                     Modelo Especial, 12 fl oz  ...               NaN
1          Run Wild Non-Alcoholic IPA, 12 fl oz  ...             fl oz
2  Blue Moon Belgian White Ale (6-pk), 12 fl oz  ...             fl oz
3                 Modelo Especial 6pk, 12 fl oz  ...               NaN
4                 Blood Orange Mint 6pk, 355 ml  ...               can

[5 rows x 10 columns]
D

In [3]:
# some weird thing is happening with the replace string at line 19, don't have time to check it 